---
title: "TIL from Stanford CME295 Transformers & LLMs | Lecture 8 - LLM Evaluation"
date: "2025-12-26"
categories: 
    - LLM Evaluation
---

Output quality
- instruction following
- coherence
- factuality


System performance
- latency
- cost
- reliability


- human ratings are gold standard. 
    - Subjectivity of human ratings -> agreement rate = $p_{o}$ -> o stands for observed
        - "How much better is our agreement than what we’d expect just by
    chance, given how the raters actually use the categories?" "A coefficient of agreement for nominal scales", Cohen, 1960, "Measuring nominal scale agreement among many raters", Fleiss, 1971.
        - Variants. Cohen's Kappa, Fleiss' Kappa, Krippendorff's alpha

    - human ratings are slow and expensive
- rule-based metrics
    - METEOR. Metric for Evaluation of Translation with Explicit ORdering
        - write labels once, use as reference
    - BLEU (BiLingual Evaluation Understudy)
    - ROUGE (Recall-Oriented Understudy for Gisting Evaluation)
        - Many variants (ROUGE-N, ROUGE-L, etc.) with suite of metrics
    - Problem with rule-based: 
        - Does not take into account stylistic variations
        - Correlation with human rating is not that great
        - still requires human ratings

- LLM-as-a-Judge
    -  Use LLM to rate quality of response.
    - <prompt, model-response, criteria> -> <score (pass/fail), reason>
    - Structured Outputs (in practice)
    - Benefit: No need for reference / label; Interpretability via rationales
    - Variations: Point-wise or Pair-wise (A or B is better)
        - Position A - position bias    
            - Remedy. "Take the average", or tweak position embeddings
        - Verbosity bias
            - Remedy. Explicit guidelines, few-shot, and/or penalty on output length
        - Self-enhancement bias
            - Remedy, Don't use agent as judge

    - Best pracitices
        - crisp guidelines
        - binary scale over granular ones
        - write rationaile before outputting score (autoregressive)
        - mitigate biases (position, verbosity, self-enhancement)
        - calibrate with human judgements
        - low temperature for reproducibility


- Typical things we want to evaluate for 
    - task performance
        - Usefulness
        - Factuality
        - Relevance
    - alignment
        - tone
        - style
        - safety

- Factuality
    - quantify factuality - president roosevelt and teddy example
    - decompose into various facts and do weighted average

- ReAct
    - Tool prediction error 
        - LLM directly issues reposnse -> tool router error -> retrain tool router
        - LLM directly issues response -> tool found but model doesn't know how to use it -> SFT train model or better adjust prompt associated to target API
        - hallucinates tool? -> llm using tool that doesn't exist -> perhaps model too weak -> upgrade model
            - or API naming is not logical -> revamp API
            - or instructions are unclear -> revamp top-level instructions

        - LLM uses wrong tool 
            - tool router error -> retrain tool router
            - model chose wrong tool -> SFT train or better adjust prompt associated to target API
        
        - Infers wrong argument:
            - argument can't be inferred -> Introduce a helper tool and/or ensure the context carries the right information
            - Model doesn't know how to use the tool -> SFT train or better adjust prompt associated to target API  
    - Tool call errors
        - wrong response or error returned
            - sometimes, errors are legitimitate
            - otherwise, fix the tool implementation
        - no response
            - when final response is hallucinated, we often see this
            - Return something, even if it's an empty JSON; in general, return meaningful tool outputs
    - Response generation error: wrong response
        - Final response doesn't convey the tool's response
            - model lacks grounding capabilities - upgrade LLM
            - Tool response spams the context window - Trim/summarize information returned by the backend
            - Tool response does not convey information meaningfully - Make the tool output format descriptive

Summary        
- Failures: does not use/hallucinates/uses the wrong tool, infers wrong argument
- Failures: wrong response, no response (tool impl.)
- Failure: wrong response (generation)

Modeling
- Weak reasoning, grounding capabilities
- Too much going on in the context window
- Tool modeling isn't right
Tool
- Tool itself has a problem
- Output of the tool isn't interpretable


Common benchmarks
- MMLU for knowledge - Massive Multitask Language Understanding  - MCQ
• Accurately state
facts about the world
• More "breadth" than
"depth"
• Reflects pretraining
quality

- Reasoning benchmarks
    - AIME (American Invitational Mathematics Examination)
        - output correct answer on ~30 math problems Geometry, Algebra, Analysis
    - PIQA (Physical Interaction: Question Answering )
        - Characteristics. 2 possible choices per question on everyday situations
anchored in Physics.
Has approximately 20,000 examples.
Evaluation criteria. Find the right choice among Sol1/Sol2.


- Coding
    - SWE bench (SoftWare Engineering benchmark)
        - • Generate
syntactically correct
code
• Tests for
programming
proficiency
• Proxy for tool use
abilities

Characteristics. 2,294 software engineering problems from real GitHub issues
across 12 popular Python repositories.
Each problem has:
● a base commit
● an already merged PR with tests


Evaluation criteria. Generated PR passes all test cases.



- Safety
- Prevent harmful,
toxic, inappropriate
behavior
• Surfaces
vulnerabilities before
deployment
• Alignment with
custom preferences

- HarmBench (Harmful Behavior Benchmark)
Characteristics. 510 unique harmful behaviors (400 text-based, 110 multimodal)
split into:
● "Standard"
● "Copyright"
● "Contextual"
● "Multimodal"
Criteria: violate laws and widely-held norms
Evaluation criteria. Attack success rate (ASR).

- classifier as output

- there are both contextual as well as multi-modal behaviors


- Agents
  - 𝜏-bench = Tool-Agent-User Interaction Benchmark
  - Characteristics. A given set of database schema, APIs and policies across 2
domains:
● Airline agent
○ 500 users, 300 flights, 2000 reservations
○ ~10 tools and 50 tasks
● Retail agent
○ 500 users, 50 products, 1000 orders
○ ~10 tools and 115 tasks
Evaluation criteria. Maximize reward and pass^k


- Pass^k = "Probability that all k attempts succeed"



Role of benchmarks.
● A projection of performance across a given axis
● Different models may be good at different
things

Definition. Pareto curve = set of solutions that "optimizes" a trade-off.
https://winston-bosan.github.io/llm-pareto-frontier/

Possible trade-offs.
● quality vs. cost/latency
● quality vs. safety
● quality vs. context length

---
Beware of data contamination
Problem. Benchmark's clues may be contained in the training set

Precautions.
● Use an identifier such as a hash
● For tools, use a blocklist
● Evaluate on newer test versions!

Lessons.
● Should not over-index on benchmarks
● Need for ~organic perspectives to complete the picture: Chatbot Arena
● ...just try a few models out yourself!



>**Licensing Notice**: Text and media: [CC BY 4.0](https://creativecommons.org/licenses/by/4.0/); Code: [Apache License 2.0](http://www.apache.org/licenses/LICENSE-2.0)